In [9]:
q = 'whats your name?'
rewrite_answers = ['ethan', 'aiden', 'anita', 'ethan is ']
prompt_q = 'Q: ' + q

ans2id = {}
cleaned_answers = []
for cdt_ans in rewrite_answers:
    if cdt_ans.startswith('A: '):
        cdt_ans = cdt_ans[len('A: '):]
    cleaned_answers.append(cdt_ans)
    if cdt_ans not in ans2id: ans2id[cdt_ans] = len(ans2id)

for ans_id, cdt_ans in enumerate(list(ans2id.keys())):
    prompt_q = prompt_q + f'\nA{ans_id+1}: ' + cdt_ans
print(prompt_q)
prev_ans_set='l'
prompt_q = prompt_q + f'\nAnswer set at the begining: {prev_ans_set}'
user_prompt = prompt_q
user_prompt

Q: whats your name?
A1: ethan
A2: aiden
A3: anita
A4: ethan is 


'Q: whats your name?\nA1: ethan\nA2: aiden\nA3: anita\nA4: ethan is \nAnswer set at the begining: l'

In [19]:

from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import torch
SapBERT_tokenizer = AutoTokenizer.from_pretrained('cambridgeltl/SapBERT-from-PubMedBERT-fulltext')
SapBERT_model = AutoModel.from_pretrained('cambridgeltl/SapBERT-from-PubMedBERT-fulltext')


In [23]:
def get_emb(names):
    encoded_input = SapBERT_tokenizer(names, padding=True, truncation=True, return_tensors='pt')
    # Compute token embeddings
    with torch.no_grad():
        model_output = SapBERT_model(**encoded_input)

    # Perform pooling. In this case, mean pooling.
    sentence_embedding = model_output.last_hidden_state.mean(dim=1)
    return sentence_embedding

In [34]:
names = ['ethan', 'anita', 'aiden', 'ethan']

In [35]:
emb = get_emb(names)
emb

tensor([[ 0.3823,  0.1725, -0.1433,  ...,  0.1912,  1.4871, -0.1506],
        [-0.2917, -0.7279, -0.2811,  ...,  0.3248,  1.0667,  0.3110],
        [-0.7845,  0.0559, -0.0715,  ...,  0.6758, -0.0391, -0.1651],
        [ 0.3823,  0.1725, -0.1433,  ...,  0.1912,  1.4871, -0.1506]])

In [36]:
similarity_score = cosine_similarity(emb)#[0][0]
similarity_score

array([[1.0000002 , 0.4950865 , 0.4311273 , 1.0000002 ],
       [0.4950865 , 1.        , 0.41205305, 0.49508664],
       [0.4311273 , 0.41205305, 1.0000002 , 0.4311274 ],
       [1.0000002 , 0.49508664, 0.4311274 , 1.0000002 ]], dtype=float32)

In [45]:
from collections import Counter

# Given list of names, possibly with duplicates
names = ['ethan', 'anita', 'aiden', 'adam', 'ethan', 'ethan c', 'john', 'jack']

# Assuming `get_emb` function is defined as before and returns embeddings
emb = get_emb(names)
similarity_score = cosine_similarity(emb)
print(similarity_score)
# Threshold for similarity
threshold = 0.98

# Group names based on similarity
groups = []
used_indices = set()

for i in range(len(names)):
    if i in used_indices:
        continue
    # This name hasn't been grouped yet, so start a new group
    current_group = [names[i]]
    used_indices.add(i)
    for j in range(i + 1, len(names)):
        if j in used_indices:
            continue
        if similarity_score[i, j] > threshold:
            current_group.append(names[j])
            used_indices.add(j)
    groups.append(current_group)

    
# Calculate frequencies of each original name
name_frequencies = Counter(names)
print('name_frequencies',name_frequencies)
# Print the results
print("Groups based on similarity (>0.98):")
for i, group in enumerate(groups):
    print(f"Group {i+1}: {', '.join(group)}")

print("\nFrequencies of each name:")
for name, frequency in name_frequencies.items():
    print(f"{name}: {frequency}")


[[0.9999997  0.4949035  0.43019283 0.46856818 0.9999998  0.8340788
  0.4264168  0.36038753]
 [0.4949035  0.9999994  0.4107427  0.4855961  0.4949036  0.42507333
  0.4229035  0.37770063]
 [0.43019283 0.4107427  1.         0.46233964 0.43019277 0.42026806
  0.4626209  0.43875158]
 [0.46856818 0.4855961  0.46233964 1.0000002  0.46856827 0.38939947
  0.47662634 0.373399  ]
 [0.9999998  0.4949036  0.43019277 0.46856827 0.99999964 0.8340788
  0.42641678 0.36038762]
 [0.8340788  0.42507333 0.42026806 0.38939947 0.8340788  1.
  0.44818765 0.36909014]
 [0.4264168  0.4229035  0.4626209  0.47662634 0.42641678 0.44818765
  0.99999964 0.5341475 ]
 [0.36038753 0.37770063 0.43875158 0.373399   0.36038762 0.36909014
  0.5341475  0.99999976]]
name_frequencies Counter({'ethan': 2, 'anita': 1, 'aiden': 1, 'adam': 1, 'ethan c': 1, 'john': 1, 'jack': 1})
Groups based on similarity (>0.98):
Group 1: ethan, ethan
Group 2: anita
Group 3: aiden
Group 4: adam
Group 5: ethan c
Group 6: john
Group 7: jack

Frequen

In [46]:
def batch_forward(q: str, rewrite_answers, prev_ans_set):
    prompt_q = 'Q: ' + q

    ans2id = {}
    cleaned_answers = []
    for cdt_ans in rewrite_answers:
        if cdt_ans.startswith('A: '):
            cdt_ans = cdt_ans[len('A: '):]
        cleaned_answers.append(cdt_ans)
        if cdt_ans not in ans2id: ans2id[cdt_ans] = len(ans2id)

    for ans_id, cdt_ans in enumerate(list(ans2id.keys())):
        prompt_q = prompt_q + f'\nA{ans_id+1}: ' + cdt_ans
    prompt_q = prompt_q + f'\nAnswer set at the begining: {prev_ans_set}'
    user_prompt = prompt_q

    tokens = tokenizer.tokenize(system_prompt + user_prompt)
    print(q)
    print(len(tokens))

    _max_tokens = 2000
    curr_model = "gpt-3.5-turbo-0613"
    if len(tokens) > 2000:
        curr_model = "gpt-3.5-turbo-1106"


    print(f"Answer set at the begining: {prev_ans_set}")
    print(temperature)
    messages = format_message(system_prompt, user_prompt)
    response = completion_with_backoff(
        model=curr_model,
        messages=messages,
        temperature=temperature,
        max_tokens=_max_tokens,
        n=1,
        )
    ans_model = response['choices'][0]['message']['content']
    ext_ans, ans_sets = extract_rewrite(ans_model)

    if len(ext_ans) != len(ans2id):
        print("failed!")
        print(ans_model)

        return INVLAID_RESUTL, [], prev_ans_set

    map_back_list = []
    for ans in cleaned_answers:
        map_back_list.append(ext_ans[ans2id[ans]])
    return SUCCESS, map_back_list, ans_sets[-1]

In [48]:
 batch_forward(q, rewrite_answers, prev_ans_set)

NameError: name 'tokenizer' is not defined

In [64]:
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import torch
from collections import Counter, defaultdict

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

# Function to compute BERT embeddings
def get_bert_embeddings(sentences):
    encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        output = model(**encoded_input)
    embeddings = output.last_hidden_state.mean(dim=1)
    return embeddings

# Input list of lists
input_lists = [['ethan', 'aiden', 'anita'], ['ethan', 'USA', 'anita'], ['aiden', 'anita', 'america']]

# Flatten the list to process all items, including duplicates
all_items_flat = [item for sublist in input_lists for item in sublist]

# Get embeddings for all items, considering duplicates
all_items_embeddings = get_bert_embeddings(all_items_flat)

# Calculate cosine similarity between all items
similarity_matrix = cosine_similarity(all_items_embeddings)

# Threshold for similarity
threshold = 0.90

# Initialize groups
groups = []
item_index_group = {}

for i in range(len(all_items_flat)):
    found_group = False
    for group in groups:
        if any(similarity_matrix[i, j] > threshold for j in group):
            group.append(i)
            found_group = True
            break
    if not found_group:
        groups.append([i])

# Map group indices back to item names, considering duplicates
grouped_items_by_index = defaultdict(list)
for group in groups:
    # Use the first item in the group as the representative
    representative = all_items_flat[group[0]]
    for index in group:
        grouped_items_by_index[group[0]].append(all_items_flat[index])

item_counts = sum([len(group) for group in groups])
# Calculate frequencies of groups using the first name in each group as the key
group_frequencies = {all_items_flat[group[0]]: len(group)/item_counts for group in groups}

# Output results
print("Grouped Items based on similarity (>0.98):")
for index, items in grouped_items_by_index.items():
    print(f"Index {index}: {', '.join(items)}")

print("\nFrequencies of each group (by first item):")
for item, frequency in group_frequencies.items():
    print(f"{item}: {frequency}")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Grouped Items based on similarity (>0.98):
Index 0: ethan, ethan
Index 1: aiden, aiden
Index 2: anita, anita, anita
Index 4: USA, america

Frequencies of each group (by first item):
ethan: 0.2222222222222222
aiden: 0.2222222222222222
anita: 0.3333333333333333
USA: 0.2222222222222222


In [60]:
similarity_matrix

array([[1.0000002 , 0.864056  , 0.81072766, 1.0000004 , 0.73723435,
        0.8107277 , 0.8640561 , 0.8107277 , 0.79322857],
       [0.864056  , 0.99999994, 0.7623788 , 0.864056  , 0.7429619 ,
        0.7623787 , 1.0000001 , 0.7623788 , 0.77343696],
       [0.81072766, 0.7623788 , 1.0000002 , 0.81072783, 0.77517635,
        1.0000001 , 0.7623787 , 1.0000002 , 0.8317076 ],
       [1.0000004 , 0.864056  , 0.81072783, 1.0000002 , 0.73723435,
        0.8107277 , 0.864056  , 0.8107278 , 0.7932286 ],
       [0.73723435, 0.7429619 , 0.77517635, 0.73723435, 0.99999994,
        0.77517647, 0.7429619 , 0.7751764 , 0.90159523],
       [0.8107277 , 0.7623787 , 1.0000001 , 0.8107277 , 0.77517647,
        1.0000002 , 0.7623786 , 1.0000002 , 0.8317077 ],
       [0.8640561 , 1.0000001 , 0.7623787 , 0.864056  , 0.7429619 ,
        0.7623786 , 1.        , 0.76237863, 0.77343684],
       [0.8107277 , 0.7623788 , 1.0000002 , 0.8107278 , 0.7751764 ,
        1.0000002 , 0.76237863, 1.0000004 , 0.8317077 ],


In [68]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

# Function to compute BERT embeddings
def get_bert_embeddings(sentences):
    encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        output = model(**encoded_input)
    embeddings = output.last_hidden_state.mean(dim=1)
    return embeddings.cpu().numpy()

# Input list of lists
input_lists = [['ethan', 'aiden', 'anita'], ['ethan', 'USA', 'anita'], ['aiden', 'anita', 'america']]

# Flatten the list to process all items, including duplicates
all_items_flat = [item for sublist in input_lists for item in sublist]

# Get embeddings for all items, considering duplicates
all_items_embeddings = get_bert_embeddings(all_items_flat)

# Calculate cosine similarity between all items
similarity_matrix = cosine_similarity(all_items_embeddings)

# Group items based on similarity, considering threshold
threshold = 0.98
groups = []
for i in range(len(all_items_flat)):
    grouped = False
    for group in groups:
        if any(similarity_matrix[i][j] > threshold for j in group):
            group.append(i)
            grouped = True
            break
    if not grouped:
        groups.append([i])

# Calculate group frequencies
group_frequencies = {i: len(group) for i, group in enumerate(groups)}

# Sort groups by frequency
sorted_group_freq = sorted(group_frequencies.items(), key=lambda x: x[1], reverse=True)

# Select top 3 groups
top_3_groups = sorted_group_freq[:3]

# Display top 3 grouped names and frequencies
print("1) Top three grouped names based on frequency:")
for index, f in top_3_groups:
    print(f"Group {index + 1}: {', '.join(all_items_flat[i] for i in groups[index])}")

print("\n2) Top three frequencies:")
for _, freq in top_3_groups:
    print(freq)

# Display all grouped names and their frequencies
print("\n3) All grouped names and their frequency:")
for index, freq in sorted_group_freq:
    print(f"Group {index + 1} ({', '.join(all_items_flat[i] for i in groups[index])}): {freq}")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


1) Top three grouped names based on frequency:
3
Group 3: anita, anita, anita
2
Group 1: ethan, ethan
2
Group 2: aiden, aiden

2) Top three frequencies:
3
2
2

3) All grouped names and their frequency:
Group 3 (anita, anita, anita): 3
Group 1 (ethan, ethan): 2
Group 2 (aiden, aiden): 2
Group 4 (USA): 1
Group 5 (america): 1


In [70]:

import numpy as np

array1 = np.array([0, 1, 1])
array2 = np.array([1, 2, 3])

# Element-wise multiplication
result = array1 * array2
print(result.tolist())  # Output: [0 2 3]

[0, 2, 3]


In [74]:
ys_list = [[1,2,3], [2,'d',4]]
ys = [item  for items in ys_list for item in items]
ys

[1, 2, 3, 2, 'd', 4]

In [ ]:
select_ids = sorted(ids, key=lambda x: values[x], reverse=True)[:args.n_select_sample]